In [1]:
import os
import psycopg2
from psycopg2 import sql
import multiprocessing
import numpy as np

os.environ["PGDATABASE"] = "ernieplus"


def select_rows_with_range(start_index, end_index):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")  # Replace with your connection parameters
        cur = conn.cursor()

        # Execute the SELECT query to obtain rows within the specified range
        # cur.execute("SELECT * FROM hm31.in_edges_features_cert  ORDER BY id OFFSET %s LIMIT %s", (start_index, end_index - start_index))
        cur.execute("SELECT * FROM hm31.embedding_edge_features_dev WHERE id >= %s AND id < %s", (start_index, end_index))

    # Fetch the results
        results = cur.fetchall()

        # Print the results
        #         for row in results:
        #             print("Row:", type(row))

        return results

    except Exception as e:
        print("Error:", e)
        print(f"start {start_index} end {end_index}")

    finally:
        cur.close()
        conn.close()
        
        
rows = select_rows_with_range(0, 10);
print(type(rows), len(rows))

<class 'list'> 9


In [2]:
def process_embedding(row):
    first_emb = row[3]
    second_emb = row[4]

    if type(first_emb) == list and len(first_emb) > 100:
        f = np.asarray(first_emb)
        
        if type(second_emb) == list and len(second_emb) > 100:
            s = np.asarray(second_emb)
            return np.dot(f,s)/(np.linalg.norm(f)*np.linalg.norm(s))
    
    return 0.5

In [3]:
def process_batch(rows):
    results = []
    for row in rows:
        first_node = row[1]
        second_node = row[2]
        
        cosine = process_embedding(row)
        
        results.append((first_node, second_node, cosine))
    
    return results     
process_batch(rows)

[(8355366, 2850621, 0.6374324396418246),
 (331900, 736325, 0.8036565537253293),
 (7987469, 1752549, 0.6906632200399174),
 (1285882, 509191, 0.7826393349096715),
 (442707, 319767, 0.8312524571505582),
 (115231, 505007, 0.8237135141774974),
 (6809889, 47786, 0.723653635601731),
 (567435, 708907, 0.7453958169641854),
 (6737207, 427521, 0.7322285380521133)]

In [4]:
# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"


# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

def insert_values_into_table(values_list):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")
        cur = conn.cursor()

        
        args = ','.join(cur.mogrify("(%s, %s, %s)", i).decode('utf-8') for i in values_list)
                
        cur.execute("INSERT INTO hm31.embedding_edge_weights_cert VALUES " + (args))

        conn.commit()
#         print('success')

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

In [5]:
def get_table_count():
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")  # Replace with your connection parameters
        cur = conn.cursor()

        # Execute the SELECT COUNT(*) query
        cur.execute("SELECT COUNT(*) FROM hm31.embedding_edge_features_dev")
        
        # Fetch the result
        result = cur.fetchone()

        # Print the result
        return result[0]

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()
count = int(get_table_count())
print(count)

10000


In [6]:
def insert_wrapper(start_index, end_index):
    rows = select_rows_with_range(start_index, end_index)
    results = process_batch(rows)
    insert_values_into_table(results)
    

In [7]:
from tqdm import tqdm

SPLIT_LENGTH = 100
count = 9919
CHUNCKS = count // SPLIT_LENGTH + 1

for i in tqdm(range(CHUNCKS)):
    rows = select_rows_with_range(i*SPLIT_LENGTH, min((i+1)* SPLIT_LENGTH, count + 1))
    results = process_batch(rows)
    insert_values_into_table(results)
    


  0%|          | 0/100 [00:00<?, ?it/s]

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(8355366, 2850621) already exists.


  2%|▏         | 2/100 [00:00<00:06, 14.18it/s]

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(6814646, 361487) already exists.
Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(7387432, 2047912) already exists.


  4%|▍         | 4/100 [00:00<00:06, 13.89it/s]

...

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(6121415, 319780) already exists.
Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(14044464, 446839) already exists.


 98%|█████████▊| 98/100 [00:06<00:00, 14.40it/s]

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(12949184, 387260) already exists.


100%|██████████| 100/100 [00:06<00:00, 14.30it/s]

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(5426920, 242878) already exists.

Error: duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(5174450, 593944) already exists.


In [8]:
from tqdm import tqdm



for i in tqdm(range(44500000, count)):
    rows = select_rows_with_range(i,i+1)
    results = process_batch(rows)
    insert_values_into_table(results)

0it [00:00, ?it/s]


In [9]:
import multiprocessing

lst = []

SPLIT_LENGTH = 1000
CHUNCKS = count // SPLIT_LENGTH

for i in range(CHUNCKS):
    lst.append((i*SPLIT_LENGTH, min((i+1)* SPLIT_LENGTH, count)))
    
with multiprocessing.Pool(processes=50) as pool:
    results = pool.starmap(insert_wrapper, lst)

Error: Error:duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(3690918, 575962) already exists.

 duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(2970448, 51058) already exists.
Error:
 duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(4466413, 3867905) already exists.

Error: Error:duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(1751576, 145533) already exists.
 
duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(8723950, 420339) already exists.

Error: Error:duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(3662343, 2047912) already exists.
 Error:
duplicate key value violates unique constraint "idx_unique_pair"
DETAIL:  Key (first, second)=(8355366, 2850621) already exists.
 
duplicate key value violates unique con